In [2]:
from flask import Flask,request, render_template,jsonify
from imageai.Detection import ObjectDetection
from PIL import Image as pil
from exif import Image as es
import os
import requests
import re
import time
from openpyxl import Workbook,load_workbook
import requests
import shutil
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import math
import numpy as np
import scipy.spatial as spatial
from folium.plugins import MarkerCluster

app = Flask(__name__)


APP_ROOT = os.getcwd()
@app.route("/")
def index():
    return render_template("upload.html")


@app.route("/covidmap",methods=['GET'])
def covidmap():
    raw_data = pd.read_excel(r'C:\Users\PIYUSH\New folder\zone_data.xlsx')
    ds = raw_data
    m = folium.Map(location=[18.5195700, 73.8553500],zoom_start = 14)
    mc = MarkerCluster()
    for row in raw_data.itertuples():
        if row.patients >= 10:
            mc.add_child(folium.Circle(location=[row.latitude,  row.longitude],popup=row.zone,radius =200,fill_color="#9A0C0C",line_color='#9A0C0C',fill_opacity=0.7,fill=True))
        elif row.patients < 10 and row.patients >= 5:
            mc.add_child(folium.Circle(location=[row.latitude,  row.longitude],popup=row.zone,radius =200,fill_color="#DE3737",line_color='#DE3737',fill_opacity=0.7,fill=True))
        else:
            mc.add_child(folium.Circle(location=[row.latitude,  row.longitude],popup=row.zone,radius =200,fill_color="#FDBEBE",line_color='#FDBEBE',fill_opacity=0.7,fill=True))
   
    m.add_child(mc)
    m.save('patients.html')
    print("Done!")
    source = "C:/Users/PIYUSH/New folder/patients.html"
    dest = "C:/Users/PIYUSH/New folder/templates/patients.html"
    shutil.move(source,dest)
    
    return render_template("patients.html")




@app.route("/count",methods=['GET'])
def tracker():
    
    def dist_to_arclength(chord_length):
        central_angle = 2*np.arcsin(chord_length/(2.0*R)) 
        arclength = R*central_angle
        return arclength

    co_ordinates = request.get_json()
    lat2 = float(request.args.get('lat'))
    lon2 = float(request.args.get('lng'))
    file = 'zone_data.xlsx'
    z_data = pd.read_excel(file,sheet_name='Sheet1')
    R=6371
    data = z_data
    data=data.drop(columns=['zone','patients','Sr No'])
    
    phi = np.deg2rad(data['latitude'])
    theta = np.deg2rad(data['longitude'])
    
    data['x'] = R * np.cos(phi) * np.cos(theta)
    data['y'] = R * np.cos(phi) * np.sin(theta)
    data['z'] = R * np.sin(phi)
    
    data=data.drop(columns=['latitude','longitude'])
    
    ph = np.deg2rad(lat2)
    th = np.deg2rad(lon2)
    
    x=R * np.cos(ph) * np.cos(th)
    y=R * np.cos(ph) * np.sin(th)
    z=R * np.sin(ph)
    
    array = []
    array.append(x)
    array.append(y)
    array.append(z)
    
    np_array=np.asarray(array)
    reshaped_array = np_array.reshape(1,-1)
    
    tree = spatial.KDTree(data[['x', 'y','z']])
    distance, index = tree.query(reshaped_array,k=1)
    
    dist =dist_to_arclength(distance)
    
    dist = round(dist[0],2)
    
    nearest_zone =z_data.loc[index[0],"zone"]
    

    
    return jsonify(containment_zone = nearest_zone , distance=dist)
    
    
    

@app.route("/image", methods=['POST'])
def upload():
    target = os.path.join(APP_ROOT, 'files')
    print(target)

    if not os.path.isdir(target):
        os.mkdir(target)

    for file in request.files.getlist("image"):
        print(file)
        my_image = es(file)
        aadhar = request.form['ad']
        number = request.form['mo']
        
        long = (float(my_image.gps_longitude[0]) + float(my_image.gps_longitude[1])/60 + float(my_image.gps_longitude[2])/(60*60)) * (-1 if my_image.gps_longitude_ref in ['W', 'S'] else 1)
        lat = (float(my_image.gps_latitude[0]) + float(my_image.gps_latitude[1])/60 + float(my_image.gps_latitude[2])/(60*60)) * (-1 if my_image.gps_latitude_ref in ['W', 'S'] else 1)
        
        print("Latitude = "+ str(lat))
        print("Longitude = "+ str(long))
        time = my_image.datetime.split()[1]
        date = my_image.datetime.split()[0]
        print(date)
        print(time)
        print(number)
        print(aadhar)
        image_name = "img-"+str(aadhar)+".jpg"
        
        location_request = requests.get("https://revgeocode.search.hereapi.com/v1/revgeocode?at={lat},{log}&apikey=hVlbuiVbXyZymQt5-BsSt1wmHpHiDtarbG5njDu_okQ".format(lat=lat,log=long))
        location_json = location_request.json()
        postCode = location_json['items'][0]['address']['postalCode']
        address = location_json['items'][0]['address']['label']
        print(address)
        print(postCode)
        
        
        detector = ObjectDetection()
        detector.setModelTypeAsRetinaNet()
        detector.setModelPath( os.path.join(APP_ROOT , "retinanet.h5"))
        detector.loadModel()
        detections = detector.detectObjectsFromImage(input_image=file, output_image_path=os.path.join(APP_ROOT , "imagenew.jpg"), minimum_percentage_probability=30)
        count=0
        for i in range(len(detections)):
            object = detections[i]['name']
            if object == 'person':
                count = count+1
        print(count)
        
        if(count < 1):
            return render_template("invalid.html")
        path = target+"\\"+str(postCode)
        if not os.path.isdir(path):
            os.mkdir(path)
            print(path)
        
            
        
        dest = "C:/Users/PIYUSH/New folder/files/"+str(postCode)+"/"+image_name
        save_image = pil.open(file)
        save_image.load()
        save_image = save_image.save(dest)
        
        wb=Workbook()
        filepath="C:/Users/PIYUSH/New folder/image_data.xlsx"
        wb = load_workbook(filepath)
        ws = wb.active
        ws.append([image_name,aadhar,number,address,postCode,date,time])
        wb.save(filepath)

    return render_template("complete.html")

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/May/2020 13:06:36] "GET /count?lat=18.6735871&lng=73.83368879999999 HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2020 13:06:45] "GET /covidmap HTTP/1.1" 200 -


Done!


127.0.0.1 - - [24/May/2020 13:07:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2020 13:07:46] "GET /favicon.ico HTTP/1.1" 404 -


C:\Users\PIYUSH\New folder\files
<FileStorage: '15903058803886048465894159682694.jpg' ('image/jpeg')>
Latitude = 18.671320972222222
Longitude = 73.83363577777777
2020:05:24
13:08:15
0987654321
1234567890
Borade Wadi-Jadhavwadi, Moshi, Chinchwad 412105, India
412105


tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

1


...

127.0.0.1 - - [24/May/2020 15:14:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/May/2020 15:14:28] "GET /count?lat=18.5253074&lng=73.83066509999999 HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2020 15:14:39] "GET /covidmap HTTP/1.1" 200 -


Done!


127.0.0.1 - - [24/May/2020 15:14:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/May/2020 15:24:23] "GET /count?lat=18.525293200000004&lng=73.8298463 HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2020 15:24:30] "GET /covidmap HTTP/1.1" 200 -


Done!


127.0.0.1 - - [24/May/2020 15:24:56] "GET / HTTP/1.1" 200 -


C:\Users\PIYUSH\New folder\files


'C:\\Users\\PIYUSH\\New folder\\files'

C:\Users\PIYUSH\New folder\files


C:\Users\PIYUSH\New folder\files\411016


'C:\\Users\\PIYUSH\\New folder\\files'